In [1]:
# pip install minsearch

import minsearch

In [2]:
!wget https://github.com/DataTalksClub/llm-zoomcamp/raw/refs/heads/main/01-intro/documents.json

--2025-07-02 22:50:55--  https://github.com/DataTalksClub/llm-zoomcamp/raw/refs/heads/main/01-intro/documents.json
Resolving github.com (github.com)... 20.26.156.215
Connecting to github.com (github.com)|20.26.156.215|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json [following]
--2025-07-02 22:50:55--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.005s  

2025-07-02 22:50:55 (116 MB/s) - ‘documents.jso

In [74]:
import json

In [75]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [76]:
documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [77]:
index = minsearch.Index(
    text_fields = ["question", "text", "section"],
    keyword_fields =["course"]
)

In [78]:
index.fit(documents)

In [79]:
import os
from google import genai
from dotenv import load_dotenv

load_dotenv()
gemini_key = os.getenv("GEMINI_API_KEY")
client = genai.Client(api_key=gemini_key)


In [80]:
def search(query):

    boost = {'question':3.0, 'section': 0.5}
    results = index.search(
        query = query,
        filter_dict = {'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results = 5, 
    ) 
    return results 


In [85]:
def build_prompt(query, search_results):

    prompt_template = """"
    You're a course assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION. 
    If the CONTEXT doesn't contain the answer, output NONE. 
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    
    """.strip()

    context = ""
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context = context).strip()

    return prompt


    

In [82]:
def llm(prompt):
    response = client.models.generate_content(
        model='gemini-2.0-flash-001', contents=prompt
    )
    return response.text.strip()
    

In [83]:
query = 'how do I run Kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [86]:
rag("can I start the course today?")


"Yes, even if you don't register, you're still eligible to submit the homeworks. Also, you can start by installing and setting up all the dependencies and requirements, and look over the prerequisites and syllabus to see if you are comfortable with these subjects."